In [1]:
from pyspark.sql import SparkSession

#Create Spark Session
spark=SparkSession.builder.appName("simple").getOrCreate()

#Read in text file from HDFS
HMEQ = spark.read.csv('hdfs://localhost:54310/user/andrew/data/HMEQ.csv', header=True)

#Register for SQL Use
HMEQ.createOrReplaceTempView("HMEQ_SQL")

HMEQ.show(10)

+---+----+-------+------+-------+------+----+-----+------+-----------+----+----+-----------+
|BAD|LOAN|MORTDUE| VALUE| REASON|   JOB| YOJ|DEROG|DELINQ|      CLAGE|NINQ|CLNO|    DEBTINC|
+---+----+-------+------+-------+------+----+-----+------+-----------+----+----+-----------+
|  1|1100|  25860| 39025|HomeImp| Other|10.5|    0|     0|94.36666667|   1|   9|       null|
|  1|1300|  70053| 68400|HomeImp| Other|   7|    0|     2|121.8333333|   0|  14|       null|
|  1|1500|  13500| 16700|HomeImp| Other|   4|    0|     0|149.4666667|   1|  10|       null|
|  1|1500|   null|  null|   null|  null|null| null|  null|       null|null|null|       null|
|  0|1700|  97800|112000|HomeImp|Office|   3|    0|     0|93.33333333|   0|  14|       null|
|  1|1700|  30548| 40320|HomeImp| Other|   9|    0|     0|101.4660019|   1|   8|37.11361356|
|  1|1800|  48649| 57037|HomeImp| Other|   5|    3|     2|       77.1|   1|  17|       null|
|  1|1800|  28502| 43034|HomeImp| Other|  11|    0|     0|88.76602988|

In [2]:
hmeq1 = spark.sql('''SELECT * 
                     FROM HMEQ_SQL
                     WHERE BAD =0''')

In [3]:
hmeq1.show(10)

+---+----+-------+------+-------+-------+----+-----+------+-----------+----+----+-----------+
|BAD|LOAN|MORTDUE| VALUE| REASON|    JOB| YOJ|DEROG|DELINQ|      CLAGE|NINQ|CLNO|    DEBTINC|
+---+----+-------+------+-------+-------+----+-----+------+-----------+----+----+-----------+
|  0|1700|  97800|112000|HomeImp| Office|   3|    0|     0|93.33333333|   0|  14|       null|
|  0|2000|  64536| 87400|   null|    Mgr| 2.5|    0|     0|147.1333333|   0|  24|       null|
|  0|2300| 102370|120953|HomeImp| Office|   2|    0|     0|90.99253347|   0|  13|31.58850318|
|  0|2400|  98449|117195|HomeImp| Office|   4|    0|     0|93.81177485|   0|  13|29.68182705|
|  0|2500|   7229| 44516|HomeImp|   Self|null|    0|     0|        208|   0|  12|       null|
|  0|2500|  71408| 78600|HomeImp|ProfExe|   8|    0|     0|255.7333333|   0|  12|       null|
|  0|2900| 103949|112505|HomeImp| Office|   1|    0|     0|96.10232967|   0|  13|30.05113629|
|  0|2900| 104373|120702|HomeImp| Office|   2|    0|     0|1

### Register as Hive Table

In [7]:
from pyspark.sql import HiveContext
hive_context=HiveContext(sc)

In [8]:
hive_context.sql('''CREATE TABLE hmeq2 
             AS SELECT * 
             FROM hmeq_SQL 
             WHERE BAD = 0''')

Name: org.apache.toree.interpreter.broker.BrokerException
Message: Traceback (most recent call last):
  File "/tmp/kernel-PySpark-0aa6cc22-d440-4f8e-86ef-491a881b995c/pyspark_runner.py", line 189, in <module>
    eval(compiled_code)
  File "<string>", line 4, in <module>
  File "/home/andrew/spark/python/pyspark/sql/context.py", line 384, in sql
    return self.sparkSession.sql(sqlQuery)
  File "/home/andrew/spark/python/pyspark/sql/session.py", line 541, in sql
    return DataFrame(self._jsparkSession.sql(sqlQuery), self._wrapped)
  File "/home/andrew/spark/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 1133, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/home/andrew/spark/python/pyspark/sql/utils.py", line 69, in deco
    raise AnalysisException(s.split(': ', 1)[1], stackTrace)
AnalysisException: u'Hive support is required to use CREATE Hive TABLE AS SELECT;;\nCreateTable CatalogTable(\n\tTable: `hmeq2`\n\tCreated: Mon Jul 03 14:19:02 EDT

### Show tables

In [5]:
spark.sql("show tables").show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
|        | hmeq_sql|       true|
+--------+---------+-----------+

